## Import Packages

In [96]:
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import glob
import cv2

## Load Training Images

In [98]:
train_df = pd.read_csv("./data/train.csv").set_index("id").sample(frac=1)

X = train_df.drop("species", axis=1).values
Y = train_df["species"].values

le= preprocessing.LabelEncoder()
le.fit(Y)
# Y = keras.utils.to_categorical(le.transform(Y), num_classes=99)
Y = le.transform(Y)
# Y = keras.utils.to_categorical(Y, num_classes=99)

In [99]:
train_df.head()

,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
1300,Viburnum_x_Rhytidophylloides,0.039062,0.058594,0.017578,0.013672,0.005859,0.093750,0.046875,0.000000,0.001953,...,0.000000,0.000000,0.000000,0.132810,0.020508,0.002930,0.0,0.083008,0.025391,0.000000
454,Arundinaria_Simonii,0.001953,0.080078,0.000000,0.023438,0.000000,0.000000,0.000000,0.005859,0.005859,...,0.068359,0.032227,0.002930,0.000000,0.002930,0.083984,0.0,0.004883,0.000000,0.000977
1532,Salix_Intergra,0.025391,0.007812,0.056641,0.019531,0.017578,0.007812,0.062500,0.000000,0.000000,...,0.043945,0.000000,0.036133,0.000000,0.000000,0.000000,0.0,0.000000,0.004883,0.054688
69,Quercus_Coccifera,0.001953,0.001953,0.072266,0.023438,0.023438,0.056641,0.041016,0.000000,0.000000,...,0.000000,0.000000,0.005859,0.052734,0.003906,0.153320,0.0,0.069336,0.000000,0.000000
899,Quercus_Dolicholepis,0.027344,0.087891,0.062500,0.000000,0.001953,0.107420,0.019531,0.000000,0.003906,...,0.000000,0.000000,0.000000,0.091797,0.000000,0.026367,0.0,0.353520,0.012695,0.000000


In [100]:
## Load images first
numImages = len(glob.glob('./images/*jpg'))
images = [None for i in xrange(numImages)]
cursry = [[] for i in xrange(numImages)]
for fileName in glob.glob('./images/*jpg'):
    fileNum = int(fileName[9:][:-4])
    img = cv2.imread(fileName, 0)
    
#     print(fileName, img.shape)

    cursry[fileNum-1].extend([img.shape[0], img.shape[1], img.shape[0] > img.shape[1]])

    
    img = cv2.resize(img, (64, 64))
    img = 2 * (np.float32(img) / 255 - 0.5)
    img = img - np.mean(img, 0)
    img = img.reshape((64, 64, 1))
    images[fileNum-1] = img
    
images = np.array(images)

# Load csv data next
train_images = np.array([images[int(iden)-1] for iden in train_df.index.values])


train_df["img_x"] = np.array([cursry[int(iden)-1][1] for iden in train_df.index.values])
train_df["img_y"] = np.array([cursry[int(iden)-1][0] for iden in train_df.index.values])
train_df["img_p"] = np.array([cursry[int(iden)-1][2] for iden in train_df.index.values])
X = train_df.drop("species", axis=1).values

del images

In [101]:
train_df.head()

,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture58,texture59,texture60,texture61,texture62,texture63,texture64,img_x,img_y,img_p
id,,,,,,,,,,,,,,,,,,,,,
1300,Viburnum_x_Rhytidophylloides,0.039062,0.058594,0.017578,0.013672,0.005859,0.093750,0.046875,0.000000,0.001953,...,0.132810,0.020508,0.002930,0.0,0.083008,0.025391,0.000000,641,338,False
454,Arundinaria_Simonii,0.001953,0.080078,0.000000,0.023438,0.000000,0.000000,0.000000,0.005859,0.005859,...,0.000000,0.002930,0.083984,0.0,0.004883,0.000000,0.000977,1392,208,False
1532,Salix_Intergra,0.025391,0.007812,0.056641,0.019531,0.017578,0.007812,0.062500,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.004883,0.054688,634,264,False
69,Quercus_Coccifera,0.001953,0.001953,0.072266,0.023438,0.023438,0.056641,0.041016,0.000000,0.000000,...,0.052734,0.003906,0.153320,0.0,0.069336,0.000000,0.000000,185,264,True
899,Quercus_Dolicholepis,0.027344,0.087891,0.062500,0.000000,0.001953,0.107420,0.019531,0.000000,0.003906,...,0.091797,0.000000,0.026367,0.0,0.353520,0.012695,0.000000,411,652,True


In [102]:
# train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.2)

## Logically Partition Data

In [103]:
train_X, test_X =  X[:int(len(X)*.7)], X[int(len(X)*.7):]
train_X_i, test_X_i = train_images[:int(len(X)*.7)], train_images[int(len(X)*.7):]
train_Y, test_Y =  Y[:int(len(Y)*.7)], Y[int(len(Y)*.7):]

## Build Functional Keras Model

In [104]:
def buildModel():
    conv_in = keras.layers.Input(shape=(64, 64, 1), name="conv_input")
    feat_in = keras.layers.Input(shape=(195,), name="feat_input")

    x = keras.layers.Conv2D(4, kernel_size=(3,3))(conv_in)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=(2,2))(x)
    x = keras.layers.Dropout(0.3)(x)
    
    x = keras.layers.Conv2D(16, kernel_size=(3,3))(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=(2,2))(x)
    x = keras.layers.Dropout(0.3)(x)    
    
    x = keras.layers.Flatten()(x)
        
    x = keras.layers.concatenate([x, feat_in])

    x = keras.layers.Dense(1024, activation='relu')(x)
    x = keras.layers.Dense(512, activation='relu')(x)
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dropout(0.3)(x) 
    output = keras.layers.Dense(99, activation='softmax')(x)
    
    model = keras.models.Model(inputs=[conv_in, feat_in], outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.categorical_accuracy])
    return model
    
buildModel().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv_input (InputLayer)         (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 62, 62, 4)    40          conv_input[0][0]                 
__________________________________________________________________________________________________
activation_51 (Activation)      (None, 62, 62, 4)    0           conv2d_51[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_51 (MaxPooling2D) (None, 31, 31, 4)    0           activation_51[0][0]              
__________________________________________________________________________________________________
dropout_80

In [105]:
predictor = KerasClassifier(build_fn=buildModel, epochs=50, batch_size=10)
predictor.fit([train_X_i, train_X], train_Y)

Epoch 1/50
693/693 [==============================] - 6s 8ms/step - loss: 15.9972 - categorical_accuracy: 0.0072
Epoch 2/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_accuracy: 0.0087
Epoch 3/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_accuracy: 0.0087
Epoch 4/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_accuracy: 0.0087
Epoch 5/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_accuracy: 0.0087
Epoch 6/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_accuracy: 0.0087
Epoch 7/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_accuracy: 0.0087
Epoch 8/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_accuracy: 0.0087
Epoch 9/50
693/693 [==============================] - 4s 6ms/step - loss: 15.9785 - categorical_

## Predict Accuracy and Export To File

In [44]:
100*accuracy_score(test_Y, predictor.predict([test_X_i, test_X]))

67.741935483870961

In [ ]:
test_df = pd.read_csv("./data/test.csv")
X_test = test_df.drop("id", axis=1)
ids_test = test_df["id"].values

In [ ]:
def generateSubmission(ids, test, model, num_classes):
    num_test = len(test)
    predictions = model.predict(test)
    confidence = model.predict_proba(test)
    output = np.zeros((num_test, num_classes+1))
    
    for i in xrange(num_test):
        p = predictions[i]
        c = confidence[i][p]
        #prob = c
        prob = min(max(10e-15, c), 1-10e-15)
        logLoss = np.log(prob)
        output[i][p+1] = -logLoss / num_test
        output[i][0] = ids[i]
    return output

out_file = generateSubmission(ids_test, X_test, predictor, 99)
headerRow=np.array(['id'] + le.inverse_transform(range(99)).tolist())
df = pd.DataFrame(data=out_file, columns = headerRow)
df['id'] = df['id'].astype(np.int)
df=df.set_index('id')
#print df.head()
# np.set_printoptions(threshold=np.inf)
# print out_file
df.to_csv('output/11_16_18_1.csv')